# Bloom Filter

In [2]:
from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


In [3]:
import string
import random
import timeit

# generate a random string of given length
def generate_random_string(length):

    letters = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    return ''.join(random.choice(letters) for _ in range(length))

# measure time to insert a elements to a set
def insert_time_calc(currentStruct, insert_data):
    insert_time = timeit.timeit(lambda: [currentStruct.insertElement(element) for element in insert_data], number=1)
    
    return insert_time

# measure time to search for a elements in a set
def search_time_calc(currentStruct, search_data):
    search_time = timeit.timeit(lambda: [currentStruct.searchElement(element) for element in search_data], number=1)
    
    return search_time

def false_probability_calculator(n, m, k):
    return (1 - (2.718281828459045 ** (-1 * ((k * n) / m)))) ** k


In [4]:
from bitarray import bitarray
        
class BloomFilterSet(AbstractSet):
    def __init__(self):
        self.N = 1_000_000
        self.arr = bitarray(self.N)
        self.arr.setall(0)
        
    
    # uses mid-square hashing method
    def hashFunc1(self, data):
        total = 0
        count = 1
        for ch in data:
            total += ord(ch) * count
            count *= 10
        keyLen = len(str(total))
        totalStr = str(total * total)
        newKey = totalStr[len(totalStr)//2 - keyLen//2:len(totalStr)//2 + (keyLen - keyLen//2)]
        return int(newKey) % self.N

    
    # uses string folding
    def hashFunc2(self, data):
        total = 0
        count = 1
        if len(data) < 4:
            for ch in data:
                total = ord(ch) * count
                count *= 10
        else:
            times = len(data)//4
            rem = len(data) % 4
            index = 0
            for i in range(1, times + 1):
                tempTotal = 0
                for j in range(index, i*4):
                    tempTotal = ord(data[j]) * count
                    count *= 10
                total += tempTotal
                index += i*4
            count = 10
            for ch in data[(len(data)-rem):len(data)]:
                total += ord(ch) * count
                count *= 100
        return total % self.N

        
    def insertElement(self, element):
        inserted = False
        
        hash1 = self.hashFunc1(element)
        hash2 = self.hashFunc2(element)
        if self.arr[hash1] == 0:
            self.arr[hash1] = 1
            inserted = True
        if self.arr[hash2] == 0:
            self.arr[hash2] = 1
        if self.arr[hash1] == 0 and self.arr[hash2] == 0:
            inserted = True

        return inserted
    
    
    def searchElement(self, element):     
        found = True

        hash1 = self.hashFunc1(element)
        hash2 = self.hashFunc2(element)
        if self.arr[hash1] != 1:
            found = False
        elif self.arr[hash2] != 1:
            found = False
        
        return found   


In [5]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    def __init__(self):        
        pass           


    # generate a list of length size with random strings (of length up to 100)
    # for search and insertion testing
    def generateData(self, size, sort=False):
        data = []

        # generate the list random strings of length 3 to 10
        for _ in range(size):
            data.append(generate_random_string(random.randint(3, 10)))
            
        assert len(data) == size
        
        if sort:
            return sorted(data)
        return data


In [6]:
import timeit

# Testing all 4 data structures on synthetic data

max_size = 1_000_000

insert_size = max_size
search_size = max_size

# synthetic data generator
syn_test = TestDataGenerator()

# generate synthetic data to insert
insert_data = syn_test.generateData(insert_size)

# generate data to search (half of insert_data and half of generated data - all shuffled)
search_data = insert_data[:(len(insert_data) // 2)] + syn_test.generateData(search_size // 2)
random.shuffle(search_data)

# ensure the size of insert and search data matches
assert len(insert_data) == max_size
assert len(search_data) == max_size
currentStruct = BloomFilterSet()
print("Bloom Filter")
insert_time = timeit.timeit(lambda: [currentStruct.insertElement(word) for word in insert_data], number=1)
search_time = timeit.timeit(lambda: [currentStruct.searchElement(word) for word in search_data], number=1) 
print(f"\tTime to insert {insert_size} words: {insert_time} seconds")
print(f"\tTime to search for {search_size} words: {search_time} seconds")


Bloom Filter
	Time to insert 1000000 words: 2.7895474000000036 seconds
	Time to search for 1000000 words: 2.8003597000000013 seconds


In [ ]:
import matplotlib.pyplot as plt

# measuring time to insert/search 10 000 elements every 100 000 elements
STEP = 100_000
STEP_INSERT_SEARCH = 10_000


# numbers of elements in the set at times of measurements
nums_of_elements_insert = []
nums_of_elements_search = []
bloom_insert_times = []
bloom_search_times = []


# synthetic data generator
tdg = TestDataGenerator()
bloomSet = BloomFilterSet()

for i in range(0, 101):
    # current number of elements in the set before insertion and search
    nums_of_elements_insert.append(i * STEP)
    nums_of_elements_search.append(i * STEP + STEP_INSERT_SEARCH)

    # generate random elements to insert
    insert_elements = tdg.generateData(STEP_INSERT_SEARCH)
    assert len(insert_elements) == STEP_INSERT_SEARCH
    
    # measure time to insert random elements into the set
    bloom_insert_times.append(insert_time_calc(bloomSet, insert_elements))

    # generate elements to search (half that are in the set for sure, half random)
    search_elements = insert_elements[:(STEP_INSERT_SEARCH//2)] + tdg.generateData(STEP_INSERT_SEARCH//2)
    random.shuffle(search_elements)
    assert len(search_elements) == STEP_INSERT_SEARCH
    
    # measure time to search for the elements in the set
    bloom_search_times.append(search_time_calc(bloomSet, search_elements))
        
    # generate the rest of random elements to be inserted in the step
    random_elements = tdg.generateData(STEP - STEP_INSERT_SEARCH)
    
    # insert it into every ds (no time measuring - using insert_time_calc temporary)
    for element in random_elements:
        bloomSet.insertElement(element)

# plot search graph
plt.plot(nums_of_elements_search, bloom_search_times, label="Bloom")

plt.xlabel("Number of elements in the set")
plt.ylabel("Time to search for " + str(STEP_INSERT_SEARCH) + " elements (sec)")
plt.legend()
plt.show()

# plot insert graph
plt.plot(nums_of_elements_insert, bloom_insert_times, label="Bloom")

plt.xlabel("Number of elements in the set")
plt.ylabel("Time to insert " + str(STEP_INSERT_SEARCH) + " elements (sec)")
plt.legend()
plt.show()
